In [204]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import math
import random

In [205]:
PARAM_NAMES = [f'PARAM_{i}' for i in range(44)]
DATA_DIRECTORY = 'raw_data'

### Data load

In [206]:
data_files_list = os.listdir(DATA_DIRECTORY)[1:]
raw_data = []
for filename in data_files_list:
    raw_data.append( pd.read_csv(f'{DATA_DIRECTORY}/{filename}', sep=';') )

### Cross event analize

In [207]:
raw_data_evented_object_count = { f'event_{i}':0 for i in range(6)}
raw_data_evented_case_count = { f'event_{i}':0 for i in range(6)}
for data in raw_data:
    for i in range(6):
        len_ = len(data[data[f'Y_{i}'] == 1])
        raw_data_evented_case_count[f'event_{i}'] += len_
        if len_ :
            raw_data_evented_object_count[f'event_{i}']+=1
count_represent = pd.DataFrame()
count_represent['event']   = [i for i in range(6)]
count_represent['objects'] = list(raw_data_evented_object_count.values())
count_represent['count']   = list(raw_data_evented_case_count.values())
count_represent

,event,objects,count
0,0,99,282
1,1,97,232
2,2,162,974
3,3,6,8
4,4,65,136
5,5,160,1062


### Making dataset for case 5

In [208]:
EVENT_NAME = 'Y_5'

In [209]:
base_dataset = []
for data in raw_data:
    if len(data[data[EVENT_NAME] == 1]):
       base_dataset.append(data) 

base_dataset[0]

,DATE_ID,PARAM_0,PARAM_1,PARAM_2,PARAM_3,PARAM_4,PARAM_5,PARAM_6,PARAM_7,PARAM_8,...,PARAM_40,PARAM_41,PARAM_42,PARAM_43,Y_0,Y_1,Y_2,Y_3,Y_4,Y_5
0,0,0.842693,0.681159,0.192972,NaN,NaN,NaN,NaN,NaN,0.192972,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,1,0.756889,0.434783,0.148241,NaN,NaN,NaN,NaN,NaN,0.148241,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,2,0.814091,0.369565,0.146377,NaN,NaN,NaN,NaN,NaN,0.146377,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,3,0.685385,0.471014,0.204155,NaN,NaN,NaN,NaN,NaN,0.204155,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,4,0.099058,0.471014,0.107237,NaN,NaN,NaN,NaN,NaN,0.107237,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,84,0.313568,0.485507,0.252614,NaN,NaN,NaN,NaN,NaN,0.252614,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
85,85,0.327869,0.376812,0.282435,NaN,NaN,NaN,NaN,NaN,0.282435,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
86,86,0.499477,0.492754,0.502364,NaN,NaN,NaN,NaN,NaN,0.502364,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
87,87,0.528078,0.362319,0.500500,NaN,NaN,NaN,NaN,NaN,0.500500,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


#### NaN replace

In [210]:
REPLACE_VALUE = 0.0001
base_dataset = list(map( lambda x: x.replace(np.nan, REPLACE_VALUE).replace(0., REPLACE_VALUE), base_dataset))
Y = np.array(list(map( lambda x: x[EVENT_NAME], base_dataset)))
base_dataset = list(map( lambda x: x[PARAM_NAMES], base_dataset))
base_dataset[0].head()


,PARAM_0,PARAM_1,PARAM_2,PARAM_3,PARAM_4,PARAM_5,PARAM_6,PARAM_7,PARAM_8,PARAM_9,...,PARAM_34,PARAM_35,PARAM_36,PARAM_37,PARAM_38,PARAM_39,PARAM_40,PARAM_41,PARAM_42,PARAM_43
0,0.842693,0.681159,0.192972,0.0001,0.0001,0.0001,0.0001,0.0001,0.192972,0.880597,...,0.0001,0.907417,0.083333,0.083333,0.000100,0.0001,0.0001,0.0001,0.0001,0.0001
1,0.756889,0.434783,0.148241,0.0001,0.0001,0.0001,0.0001,0.0001,0.148241,0.791045,...,0.0001,0.787052,0.000100,0.000100,0.019233,0.0001,0.0001,0.0001,0.0001,0.0001
2,0.814091,0.369565,0.146377,0.0001,0.0001,0.0001,0.0001,0.0001,0.146377,0.850746,...,0.0001,0.000100,0.083333,0.083333,0.038467,0.0001,0.0001,0.0001,0.0001,0.0001
3,0.685385,0.471014,0.204155,0.0001,0.0001,0.0001,0.0001,0.0001,0.204155,0.716418,...,0.0001,0.987248,0.083333,0.083333,0.057671,0.0001,0.0001,0.0001,0.0001,0.0001
4,0.099058,0.471014,0.107237,0.0001,0.0001,0.0001,0.0001,0.0001,0.107237,0.089552,...,0.0001,0.852205,0.083333,0.083333,0.076844,0.0001,0.0001,0.0001,0.0001,0.0001


#### Interval values getting

In [221]:
def dataset_silicer(data:np.ndarray, slice=3):
    sliced_data = []
    for i in range(len(data),0,-1):
        sliced_data.insert(0, data[max(0, i-slice):i])
    return sliced_data
dataset_silicer(base_dataset[0][PARAM_NAMES].to_numpy())[-1].shape

(3, 44)

In [222]:
sliced_data = list(map( lambda x : dataset_silicer(x[PARAM_NAMES].to_numpy()), base_dataset))
len(sliced_data[0][0])

1

#### Sum metric

In [223]:
sum_metric = np.array([[np.sum(slice, axis=0) for slice in data] for data in sliced_data ])
sum_metric.shape

(160, 89, 44)

#### Max metric

In [224]:
max_metric = np.array([[np.max(slice, axis=0) for slice in data] for data in sliced_data ])
max_metric.shape

(160, 89, 44)

#### Jump metric

In [225]:
jump_metric = np.array([[list(np.power(np.max(slice, axis=0)/np.sum(slice, axis=0), 2)) for slice in data] for data in sliced_data ])
jump_metric.shape

(160, 89, 44)

### Dataset generation

In [226]:
Y_future = Y[:,1:]
Y_future.shape

(160, 88)

In [227]:
jump_metric_past = jump_metric[:,:-1]
max_metric_past  = max_metric [:, :-1]
sum_metric_past  = sum_metric [:,:-1]

In [228]:
X_past = np.array([jump_metric_past, max_metric_past, sum_metric_past])

In [229]:
X_past.shape, Y_future.shape

((3, 160, 88, 44), (160, 88))

#### Save data

In [230]:
import json

In [231]:
with open('x_past.json', 'w') as file:
    file.write(json.dumps(X_past.tolist()))

with open('y_future.json', 'w') as file:
    file.write(json.dumps(Y_future.tolist()))
    

In [232]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [233]:
x_train, x_test = np.concatenate(np.concatenate(X_past, axis=-1), axis=0)[:-2000], np.concatenate(np.concatenate(X_past, axis=-1), axis=0)[-2000:]
y_train, y_test = np.concatenate(Y_future, axis=0).reshape((14080, 1))[:-2000], np.concatenate(Y_future, axis=0).reshape((14080, 1))[-2000:]

In [316]:
inputs = keras.Input(shape=(132, ))
f_1 = layers.Flatten()(inputs)
d_l_1 = layers.Dense(200, activation="relu")(f_1)
d_l_2 = layers.Dense(100, activation="relu")(d_l_1)
d_l_3 = layers.Dense(20, activation="relu")(d_l_2)
out = layers.Dense(1, activation="sigmoid")(d_l_3)

In [317]:
m_1 = keras.Model(inputs=inputs, outputs=out, name="test_tf")
m_1.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [280]:
m_1.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test), )

Epoch 1/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0254 - loss: 0.0420 - val_accuracy: 0.0050 - val_loss: 0.0766
Epoch 2/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0304 - loss: 0.0436 - val_accuracy: 0.0040 - val_loss: 0.0785
Epoch 3/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0310 - loss: 0.0398 - val_accuracy: 0.0055 - val_loss: 0.0794
Epoch 4/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0343 - loss: 0.0385 - val_accuracy: 0.0085 - val_loss: 0.0814
Epoch 5/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0313 - loss: 0.0390 - val_accuracy: 0.0055 - val_loss: 0.0802
Epoch 6/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0330 - loss: 0.0400 - val_accuracy: 0.0040 - val_loss: 0.0760
Epoch 7/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0336 - loss: 0.0385 - val_accuracy: 0.0025 - val_loss: 0.0797
Epoch 8/20
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0338 - loss: 0.0392 - val_accuracy: 0.

In [324]:
res = pd.DataFrame({
    'pred': np.array([m_1.predict(data.reshape(1,132,1)) for data in x_test[:88]]).reshape(-1),
    'real': y_test[:88].reshape(-1)
    }
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━

In [326]:
res[res['real'] > 0.95]

,pred,real
24,0.000745,1.0
26,0.999998,1.0
52,1.000000,1.0


In [321]:
cut_x = []
cut_y = []
zero_count = 0
for i in range(len(x_train)):
    if y_train[i][0] > 0.5:
        cut_x.append(x_train[i])
        cut_y.append(y_train[i])
    else:
        if zero_count < 5000:
            zero_count += 1
            cut_x.append(x_train[i])
            cut_y.append(y_train[i])
            

In [322]:
cut_x = np.array(cut_x)
cut_y = np.array(cut_y)

In [323]:
m_1.fit(cut_x, cut_y, epochs=100, validation_data=(x_test, y_test), batch_size=10)

Epoch 1/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0807 - loss: 0.1245 - val_accuracy: 0.0135 - val_loss: 0.1357
Epoch 2/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0752 - loss: 0.0841 - val_accuracy: 0.0185 - val_loss: 0.1569
Epoch 3/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0814 - loss: 0.0712 - val_accuracy: 0.0220 - val_loss: 0.2071
Epoch 4/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0872 - loss: 0.0591 - val_accuracy: 0.0230 - val_loss: 0.2403
Epoch 5/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0944 - loss: 0.0533 - val_accuracy: 0.0210 - val_loss: 0.2169
Epoch 6/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1061 - loss: 0.0483 - val_accuracy: 0.0250 - val_loss: 0.2152
Epoch 7/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1151 - loss: 0.0464 - val_accuracy: 0.0320 - val_loss: 0.3139
Epoch 8/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1085 - loss: 0.0454 - val_accu